<a href="https://colab.research.google.com/github/Jhonier29/Proyecto-Detecci-n-de-estr-s/blob/main/CODIGOS/Clasifiacion_3_clases_Algoritmos_de_ML_ipynbt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cargar Librerias

In [ ]:
! pip install eli5

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import tensorflow as tf
from pandas import read_csv
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import permutation_importance
import eli5
from eli5.sklearn import PermutationImportance
from keras.layers import LSTM, Dense
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier

##Cargar Datos


In [ ]:
dataframe= pd.DataFrame()
backup_dataframe = dataframe
personas = ['pklS3_BD3','pklS4_BD3','pklS5_BD3','pklS6_BD3','pklS7_BD3','pklS8_BD3','pklS9_BD3','pklS10_BD3','pklS11_BD3','pklS13_BD3','pklS14_BD3','pklS15_BD3','pklS16_BD3','pklS17_BD3'] # Nombres de las bases de datos a usar
for i in personas:
  filename = '{0}.csv'.format(i)
  datos = pd.read_csv(filename,header=None)
  datos = datos.transpose()
  dataframe = pd.concat([dataframe, datos], axis=0)
dataframe= dataframe.reset_index(drop=True)
# Usar para BD1:
#nombres = ['Area' ,'COHE', 'distPKM', 'HF', 'HR' ,'LF', 'LFHF', 'pNN50', 'Ratiopoinc', 'RMSSD', 'SD1' ,'SD2', 'SDNN', 'TPower','VLF','target','target3']
# Usar para BD2:
nombres = ['Area' ,'COHE', 'distPKM', 'HF', 'HR' ,'LF', 'LFHF', 'pNN50', 'Ratiopoinc', 'RMSSD', 'SD1' ,'SD2', 'SDNN', 'TPower','VLF','GSRm','SDNNG','maxG','minG','rangoGSR','penGSR','target','target3']
# Usar para BD3:
#nombres = ['Area' ,'COHE', 'distPKM', 'HF', 'HR' ,'LF', 'LFHF', 'pNN50', 'Ratiopoinc', 'RMSSD', 'SD1' ,'SD2', 'SDNN', 'TPower','VLF','GSRm','SDNNG','maxG','minG','rangoGSR','penGSR','tempm','SDNNt','maxt','mint','RangoT','penT','target','target3']
# Usar para BD4:
#nombres = ['Area' ,'COHE', 'distPKM', 'HF', 'HR' ,'LF', 'LFHF', 'pNN50', 'Ratiopoinc', 'RMSSD', 'SD1' ,'SD2', 'SDNN', 'TPower','VLF','GSRm','SDNNG','maxG','minG','rangoGSR','penGSR','target','target3']

feature_names=nombres[0:-2]
dataframe.columns= nombres
dataframe.head()

##Normalizar datos 

In [ ]:
for names in nombres[0:-1]:
    val = dataframe[str(names)]    
    dataframe[str(names)]= (val-val.min())/(val.max()-val.min())

###Datos normalizados

In [ ]:
personas_test = ['pklS2_BD3']  #Sujeto con el cual se validara los algoritmos
testeo = pd.DataFrame()
for i in personas_test:
  filename = '{0}.csv'.format(i)
  pTest = pd.read_csv(filename,header=None)
  pTest = pTest.transpose()
  testeo = pd.concat([testeo, pTest], axis=0)
testeo= testeo.reset_index(drop=True)
testeo.columns= nombres
for names in nombres[0:-1]:
    val = testeo[str(names)]
    testeo[str(names)]= (val-val.min())/(val.max()-val.min())

##Datos de entrenamiento


In [ ]:
X=dataframe[feature_names]  # Caracteristicas
y=dataframe['target3']  # Etiqueta para clasifciacion de 3 clases

# Division de los datos para entrenamiento y prueba de manera automatica
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 80% Entrenamiento y 20% prueba
# Conjunto de datos para validar los algoritmos con el sujeto extraido
X_out= testeo[feature_names]
y_out = testeo['target3']

X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)


# Numero de divisiones de datos para validación cruzada

In [ ]:
kfold_validacion = KFold(5)

#Algoritmos de ML

## Random Forest

In [ ]:
#Hiperparámetros
n_estimators=120
max_depth=8
#Creación del modelo
RF = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
#Entrenamiento
RF.fit(X_train,y_train)
#Predicción
y_pred=RF.predict(X_test)

In [ ]:
# Metricas de analisis
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

###Caracteristicas mas importantes en RF

In [ ]:
feature_imp = pd.Series(RF.feature_importances_,index=feature_names).sort_values(ascending=False)
feature_imp
imp1= feature_imp
%matplotlib inline
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
# Reporte de datos RF
report = classification_report(y_test, y_pred, output_dict=True)
resultadosRF = pd.DataFrame(report).transpose()
resultadosRF['balanced_accuracy'] = metrics.balanced_accuracy_score(y_test,y_pred)
resultadosRF['clasificador']= 'RF'
resultados = cross_val_score(RF, X, y, cv = kfold_validacion)
resultadosRF['Cross Validation']=resultados.mean()
car_imp=feature_imp[0:len(resultadosRF.axes[0])]
resultadosRF['CAR.IMP']= car_imp.index
resultadosRF['n_extimator| maxdepth']= (str(n_estimators) +' | '+ str(max_depth)) 
resultadosRF

###Matriz de confusión

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
sns.set(font_scale=2)
sns.heatmap(conf_mat, annot=True, annot_kws={"size": 16})
plt.show()

##SVM

In [ ]:
# Hiperparámetros SVM rbf
gamma= 1
C1 = 50
# Hiperparámetros SVM polinomial
degree=5 
C2 = 50
# Hiperparámetros SVM lineal
C3 = 10
rbf_svc = svm.SVC(kernel='rbf', gamma=gamma, C=C1).fit(X_train.values, y_train)
poly_svc = svm.SVC(kernel='poly', degree=degree, C=C2).fit(X_train.values, y_train)
lin_svc = svm.LinearSVC(C=C3, max_iter=10000).fit(X_train.values, y_train)

In [ ]:
# Resultados del modelo SVM rbf
Z = rbf_svc.predict(np.c_[X_test])
conf_mat = confusion_matrix(y_test, Z)
print(conf_mat)
report = classification_report(y_test, Z, output_dict=True)
rbf = pd.DataFrame(report).transpose()
rbf['balanced_accuracy'] = metrics.balanced_accuracy_score(y_test,Z)
resultados = cross_val_score(rbf_svc, X, y, cv = kfold_validacion)
rbf['Cross Validation'] = resultados.mean()

In [ ]:
# Resultados del modelo SVM polinomial
Z = poly_svc.predict(np.c_[X_test])
conf_mat = confusion_matrix(y_test, Z)
print(conf_mat)
report = classification_report(y_test, Z, output_dict=True)
poly = pd.DataFrame(report).transpose()
poly['balanced_accuracy'] = metrics.balanced_accuracy_score(y_test,Z)
resultados = cross_val_score(poly_svc, X, y, cv = kfold_validacion)
poly['Cross Validation'] = resultados.mean()

In [ ]:
# Resultados del modelo SVM lineal
Z = lin_svc.predict(np.c_[X_test])
conf_mat = confusion_matrix(y_test, Z)
print(conf_mat)
report = classification_report(y_test, Z, output_dict=True)
lin = pd.DataFrame(report).transpose()
lin['balanced_accuracy'] = metrics.balanced_accuracy_score(y_test,Z)
resultados = cross_val_score(lin_svc, X, y, cv = kfold_validacion)
lin['Cross Validation'] = resultados.mean()

In [ ]:
rbf['C | gamma']=str(C1)+' | '+str(gamma)
poly['C | degree']= str(C2)+' | '+str(degree)
lin['C']= str(C3)

###Caracteristicas mas importantes de SVM

In [ ]:
features_svmL=pd.Series(abs(lin_svc.coef_[0]), index=feature_names).sort_values(ascending=False)
imp2=features_svmL
%matplotlib inline
car_imp=features_svmL[0:6]
lin['CAR.IMP']= car_imp.index
# Creating a bar plot
sns.barplot(x=features_svmL, y=features_svmL.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
perm_importance = permutation_importance(rbf_svc, X_test.values, y_test.values)
features = np.array(feature_names)
sorted_idx = perm_importance.importances_mean.argsort()
imp3=features[sorted_idx]
imp3=imp3[::-1]
plt.figure(figsize=(10,6))
plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

In [ ]:
rbf['CAR.IMP']= features[sorted_idx][::-1][0:6]
rbf['CAR.IMP']

In [ ]:
perm_importance = permutation_importance(poly_svc, X_test.values, y_test.values)
features = np.array(feature_names)
sorted_idx = perm_importance.importances_mean.argsort()
imp4=features[sorted_idx]
imp4=imp4[::-1]
plt.figure(figsize=(10,6))
plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

In [ ]:
poly['CAR.IMP']= features[sorted_idx][::-1][0:6]
poly['CAR.IMP']

In [ ]:
resultadosSVM = pd.concat([rbf,poly,lin], axis=0)
resultadosSVM['clasificador']= ['rbf','rbf','rbf','rbf','rbf','rbf','poly','poly','poly','poly','poly','poly','linear','linear','linear','linear','linear','linear'] 
resultadosSVM

##Red Neuronal


###3 clases

In [ ]:
# Clasificación multiclase
epochsRN =50
neurons = 0 #Neuronas de la capa oculta
# definición del modelo
def baseline_model(neurons):
  # Creación el modelo
  model = Sequential()
  model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
  #model.add(Dense(neurons, activation='relu')) #capa oculta(descomentar si se requiere)
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
estimator = KerasClassifier(build_fn=baseline_model, neurons = neurons,epochs= epochsRN, verbose=0)
seed = 5
np.random.seed(seed)
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
#Entrenamiento de la red
estimator.fit(X_train, y_train)

In [127]:
# Caracteristicas importantes
perm = PermutationImportance(estimator, random_state=1).fit(X_train, y_train)
impFNN=np.argsort(perm.feature_importances_)
features = np.array(feature_names)
imp5=features[impFNN][::-1]

In [ ]:
# Resultados RN
y_pred = estimator.predict(X_test)
report = classification_report(y_test,y_pred, output_dict=True)
resultadosRN = pd.DataFrame()
resultadosRN = pd.DataFrame(report).transpose()
resultadosRN['balanced_accuracy'] = metrics.balanced_accuracy_score(y_test,y_pred)
resultadosRN['CAR.IMP']= features[impFNN][::-1][0:6]
resultadosRN['epocas'] = str(epochsRN)
resultadosRN['clasificador'] = 'RN'
resultadosRN['Cross Validation'] = results.mean()
resultadosRN

#Red Neuronal LSTM

In [131]:
# Añadir una tercera dimensión a los conjuntos de datos
nmp=X_train.to_numpy()
Xa = np.reshape(nmp, (len(nmp),1,len(feature_names)))
XTa = np.reshape(X_test.values, (len(X_test),1,len(feature_names)))

###3 clases

In [ ]:
# Clasificación multiclase
epochsLSTM =15
# Definición del modelos
def LSTM_model():
  # Creación el modelo
  modelo = Sequential()
  modelo.add(LSTM(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
  modelo.add(Dense(3, activation='softmax'))
  modelo.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
  return modelo
estimatorLSTM = KerasClassifier(build_fn=LSTM_model,epochs= epochsLSTM, verbose=0)
seed = 5
np.random.seed(seed)
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
resultsLSTM = cross_val_score(estimatorLSTM, Xa, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (resultsLSTM .mean()*100, resultsLSTM .std()*100))

In [ ]:
# Entrenamiento de la red LSTM
estimatorLSTM.fit(Xa, y_train)

In [ ]:
# Caracterisitcas importantes
CILSTM = []
predLSTM= estimatorLSTM.predict(XTa,verbose=0).squeeze()
baseline_mae = np.mean(np.abs(predLSTM-y_test))         
COLS = list(X_train.columns)
for k in tqdm(range(len(COLS))):
                
  save_col = XTa[:,:,k].copy()
  np.random.shuffle(XTa[:,:,k])
          
  oof_preds = estimatorLSTM.predict(XTa, verbose=0).squeeze() 
  mae = np.mean(np.abs( oof_preds-y_test ))
  CILSTM.append({'feature':COLS[k],'mae':mae})
  XTa[:,:,k] = save_col

In [ ]:
print()
df = pd.DataFrame(CILSTM)
df = df.sort_values('mae',ascending=False)
imp6=np.array(df['feature'][0::])
plt.figure(figsize=(10,20))
plt.barh(np.arange(len(COLS)),df.mae)
plt.yticks(np.arange(len(COLS)),df.feature.values)
plt.title('LSTM Feature Importance',size=16)
plt.ylim((-1,len(COLS)+1))
plt.plot([baseline_mae,baseline_mae],[-1,len(COLS)+1], '--', color='orange',
          label=f'Baseline OOF\nMAE={baseline_mae:.3f}')
plt.xlabel(f'OOF MAE with feature permuted',size=14)
plt.ylabel('Feature',size=14)
plt.legend()
plt.show()

In [ ]:
#Resultados de red LSTM
predLSTM= estimatorLSTM.predict(XTa)
report = classification_report(y_test,predLSTM, output_dict=True)
resultadosRNLSTM = pd.DataFrame()
RNLSTM = pd.DataFrame(report).transpose()
RNLSTM['balanced_accuracy'] = metrics.balanced_accuracy_score(y_test,predLSTM)
resultadosRNLSTM = resultadosRNLSTM.append(RNLSTM)
resultadosRNLSTM['CAR.IMP']= np.array(df['feature'][0:6])
resultadosRNLSTM['epocas'] =str(epochsLSTM)
resultadosRNLSTM['Cross Validation'] = resultsLSTM.mean()
resultadosRNLSTM['clasificador'] = 'LSTM'
resultadosRNLSTM

ADABOOST

In [ ]:
ydf = pd.DataFrame(y_train)
ytestdf = pd.DataFrame(y_test)
# Hiperparámetros
n_estimator= 70
learning_rate= 0.1

AB = AdaBoostClassifier(n_estimators=n_estimator, learning_rate= learning_rate)
AB.fit(X_train, y_train)
y_predAB=AB.predict(X_test)

if max(y_test)==2:
  xad = X_train.drop(ydf[ydf['target3']==0].index)
  yad =y_train.drop(ydf[ydf['target3']==0].index) 
  AB2 = AdaBoostClassifier(n_estimators=n_estimator, learning_rate= learning_rate)
  AB2.fit(xad, yad)

  for i in range(0,len(y_predAB),1):
    if y_predAB[i]==1:
      y_predAB[i] = AB2.predict(X_test.loc[[i]])

print(classification_report(y_test, y_predAB))

In [ ]:
# Caracteristicas importantes
feature_impAB = pd.Series(AB.feature_importances_,index=feature_names).sort_values(ascending=False)
feature_impAB
imp7=feature_impAB
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.barplot(x=feature_impAB, y=feature_impAB.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
# Resultados AB
report = classification_report(y_test,y_predAB, output_dict=True)
resultadosAB = pd.DataFrame()
rAB = pd.DataFrame(report).transpose()
resultadosAB = resultadosAB.append(rAB)
resultadosAB['balanced_accuracy'] = metrics.balanced_accuracy_score(y_test,y_predAB)
resultadosAB['n_est | learning rate']= str(n_estimator)+' | '+str(learning_rate)
resultadosAB['clasificador']='AB'
resultados = cross_val_score(AB, X, y, cv = kfold_validacion)
resultadosAB['Cross Validation'] = resultados.mean()
car_imp=feature_impAB[0:len(resultadosAB.axes[0])]
resultadosAB['CAR.IMP']= car_imp.index
resultadosAB

### Concatenación de los resultados de los algoritmos con la división de datos 80% - 20%

In [ ]:
resul = pd.concat([resultadosRF, resultadosSVM, resultadosRN,resultadosRNLSTM, resultadosAB], axis=0)
resul.fillna('-')

### Validación con el sujeto extraido

In [ ]:
#RF
y_pred=RF.predict(X_out)
report = classification_report(y_out, y_pred, output_dict=True)
resultadosRF2 = pd.DataFrame(report).transpose()
resultadosRF2['balanced_accuracy*'] = metrics.balanced_accuracy_score(y_out,y_pred)

#SVM rbf
Z = rbf_svc.predict(np.c_[X_out])
conf_mat = confusion_matrix(y_out, Z)
print(conf_mat)
report = classification_report(y_out, Z, output_dict=True)
rbf2 = pd.DataFrame(report).transpose()
rbf2['balanced_accuracy*'] = metrics.balanced_accuracy_score(y_out,Z)

#SVM polinomial
Z = poly_svc.predict(np.c_[X_out])
conf_mat = confusion_matrix(y_out, Z)
print(conf_mat)
report = classification_report(y_out, Z, output_dict=True)
poly2 = pd.DataFrame(report).transpose()
poly2['balanced_accuracy*'] = metrics.balanced_accuracy_score(y_out,Z)

#SVM lineal
Z = lin_svc.predict(np.c_[X_out])
conf_mat = confusion_matrix(y_out, Z)
print(conf_mat)
report = classification_report(y_out, Z, output_dict=True)
lin2 = pd.DataFrame(report).transpose()
lin2['balanced_accuracy*'] = metrics.balanced_accuracy_score(y_out,Z)

resultadosSVM2 = pd.concat([rbf2,poly2,lin2], axis=0)

#RN
y_pr= estimator.predict(X_out)
report = classification_report(y_out,y_pr, output_dict=True)
resultadosRN2 = pd.DataFrame()
RNeuronal2 = pd.DataFrame(report).transpose()
RNeuronal2['balanced_accuracy*'] = metrics.balanced_accuracy_score(y_out,y_pr)
resultadosRN2 = resultadosRN2.append(RNeuronal2)

#LSTM
XTa = np.reshape(X_out.values, (len(X_out),1,len(feature_names)))
predLSTM= estimatorLSTM.predict(XTa)
report = classification_report(y_out,predLSTM, output_dict=True)
resultadosRNLSTM2 = pd.DataFrame()
RNLSTM2 = pd.DataFrame(report).transpose()
RNLSTM2['balanced_accuracy*'] = metrics.balanced_accuracy_score(y_out,predLSTM)
resultadosRNLSTM2 = resultadosRNLSTM2.append(RNLSTM2)

#AB
y_predAB=AB.predict(X_out)

if max(y_out)==2:
  xad = X.drop(ydf[ydf['target3']==0].index)
  yad =y.drop(ydf[ydf['target3']==0].index) 
  AB2 = AdaBoostClassifier(n_estimators=n_estimator, learning_rate= learning_rate)
  AB2.fit(xad, yad)
 
  for i in range(0,len(y_predAB)-1,1):
    if y_predAB[i]==1:
       y_predAB[i] = AB2.predict(X_out.loc[[i]])
    
  
report = classification_report(y_out,y_predAB, output_dict=True)
resultadosAB2 = pd.DataFrame()
rAB = pd.DataFrame(report).transpose()
resultadosAB2 = resultadosAB2.append(rAB)
    #resultadosSVM = resultadosSVM()
resultadosAB2['balanced_accuracy*'] = metrics.balanced_accuracy_score(y_out,y_predAB)
resultadosAB2[''] = ''

# Concatenación de los resultados de los algoritmos al validar con el sujeto extraido
result = pd.concat([resultadosRF2, resultadosSVM2, resultadosRN2,resultadosRNLSTM2, resultadosAB2], axis=0)
result.fillna('-')

## Procedimiento de extracción de resultados con las caracteristicas mas importantes extraidas

In [142]:
numeroCarImp=10 #Numero de caracteristicas mas relevantes con las cuales se va a entrenar
dfimp1=pd.DataFrame(imp1[0:numeroCarImp]).reset_index().rename(columns={'index':'Carac'})
dfimp2=pd.DataFrame(imp2[0:numeroCarImp]).reset_index().rename(columns={'index':'Carac'})
dfimp3=pd.DataFrame(imp3[0:numeroCarImp],columns=['Carac'])
dfimp4=pd.DataFrame(imp4[0:numeroCarImp],columns=['Carac'])
dfimp5=pd.DataFrame(imp5[0:numeroCarImp],columns=['Carac'])
dfimp6=pd.DataFrame(imp6[0:numeroCarImp],columns=['Carac'])
dfimp7=pd.DataFrame(imp7[0:numeroCarImp]).reset_index().rename(columns={'index':'Carac'})

### Lista de caracteristica más relevantes en orden de importancia

In [ ]:
dfCarImp=pd.DataFrame(pd.concat([dfimp1['Carac'],dfimp2['Carac'],dfimp3['Carac'],dfimp4['Carac'],dfimp5['Carac'],dfimp6['Carac'],dfimp7['Carac']],axis=0))
resultadosCar = dfCarImp.value_counts().reset_index()
CaracCo=list(resultadosCar['Carac'][0:numeroCarImp])
CaracCo

In [144]:
# Creación de los nuevos conjuntos de entrenamiento con las caracteristicas mas importantes
X_train2=X_train[CaracCo].copy()
X_out2=X_out[CaracCo].copy()

Entrenamiento y validación de los algoritmos

In [ ]:
#RF
RF = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
RF.fit(X_train2,y_train)
y_pred=RF.predict(X_out2)
report = classification_report(y_out, y_pred, output_dict=True)
resultadosRF3 = pd.DataFrame(report).transpose()
resultadosRF3['balanced_accuracy**'] = metrics.balanced_accuracy_score(y_out,y_pred)

#SVM 
rbf_svc = svm.SVC(kernel='rbf', gamma=gamma, C=C1).fit(X_train2.values, y_train)
poly_svc = svm.SVC(kernel='poly', degree=degree, C=C2).fit(X_train2.values, y_train)
lin_svc = svm.LinearSVC(C=C3, max_iter=10000).fit(X_train2.values, y_train)

#SVM rbf
Z = rbf_svc.predict(np.c_[X_out2])
conf_mat = confusion_matrix(y_out, Z)
print(conf_mat)
report = classification_report(y_out, Z, output_dict=True)
rbf3 = pd.DataFrame(report).transpose()
rbf3['balanced_accuracy**'] = metrics.balanced_accuracy_score(y_out,Z)

#SVM polinomial
Z = poly_svc.predict(np.c_[X_out2])
conf_mat = confusion_matrix(y_out, Z)
print(conf_mat)
report = classification_report(y_out, Z, output_dict=True)
poly3 = pd.DataFrame(report).transpose()
poly3['balanced_accuracy**'] = metrics.balanced_accuracy_score(y_out,Z)

#SVM lineal
Z = lin_svc.predict(np.c_[X_out2])
conf_mat = confusion_matrix(y_out, Z)
print(conf_mat)
report = classification_report(y_out, Z, output_dict=True)
lin3 = pd.DataFrame(report).transpose()
lin3['balanced_accuracy**'] = metrics.balanced_accuracy_score(y_out,Z)

resultadosSVM3 = pd.concat([rbf3,poly3,lin3], axis=0)

#RN
def baseline_model(neurons):
  # crea el modelo
  model = Sequential()
  model.add(Dense(X_train2.shape[1], input_dim=X_train2.shape[1], activation='relu'))
  #model.add(Dense(neurons, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  # Compile el modelo
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
estimator = KerasClassifier(build_fn=baseline_model, neurons = neurons,epochs= epochsRN, verbose=0)
estimator.fit(X_train2, y_train)
y_pr= estimator.predict(X_out2)
report = classification_report(y_out,y_pr, output_dict=True)
resultadosRN3 = pd.DataFrame()
RNeuronal3 = pd.DataFrame(report).transpose()
RNeuronal3['balanced_accuracy**'] = metrics.balanced_accuracy_score(y_out,y_pr)
resultadosRN3 = resultadosRN3.append(RNeuronal3)

#LSTM
nmp=X_train2.to_numpy()
Xa = np.reshape(nmp, (len(nmp),1,numeroCarImp))
XTa = np.reshape(X_out2.values, (len(X_out2),1,numeroCarImp))
def LSTM_model():
  # crea el modelo
  modelo = Sequential()
  modelo.add(LSTM(X_train2.shape[1], input_dim=X_train2.shape[1], activation='relu'))
  modelo.add(Dense(3, activation='softmax'))
  modelo.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
  # Compile el modelo
  return modelo
estimatorLSTM = KerasClassifier(build_fn=LSTM_model,epochs= epochsLSTM, verbose=0)
estimatorLSTM.fit(Xa, y_train)
predLSTM= estimatorLSTM.predict(XTa)
report = classification_report(y_out,predLSTM, output_dict=True)
resultadosRNLSTM3 = pd.DataFrame()
RNLSTM3 = pd.DataFrame(report).transpose()
RNLSTM3['balanced_accuracy**'] = metrics.balanced_accuracy_score(y_out,predLSTM)
resultadosRNLSTM3 = resultadosRNLSTM3.append(RNLSTM3)

#AB

AB = AdaBoostClassifier(n_estimators=n_estimator, learning_rate= learning_rate)
AB.fit(X_train2, y_train)

y_predAB=AB.predict(X_out2)

if max(y_out)==2:
  xad = X_train2.drop(ydf[ydf['target3']==0].index)
  yad =y_train.drop(ydf[ydf['target3']==0].index) 
  AB2 = AdaBoostClassifier(n_estimators=n_estimator, learning_rate= learning_rate)
  AB2.fit(xad, yad)

 
  for i in range(0,len(y_predAB)-1,1):
    if y_predAB[i]==1:
       y_predAB[i] = AB2.predict(X_out2.loc[[i]])
    
report = classification_report(y_out,y_predAB, output_dict=True)
resultadosAB3 = pd.DataFrame()
rAB = pd.DataFrame(report).transpose()
resultadosAB3 = resultadosAB3.append(rAB)
resultadosAB3['balanced_accuracy**'] = metrics.balanced_accuracy_score(y_out,y_predAB)

# Concatenación de los resultados de los algoritmos al validar con el sujeto extraido y caracteristicas reducidas
result2 = pd.concat([resultadosRF3, resultadosSVM3, resultadosRN3,resultadosRNLSTM3, resultadosAB3], axis=0)
result2.fillna('-')

##Unión de todos los resultados

In [ ]:
Final_res = pd.concat([resul,result,result2], axis=1)
Final_res.fillna('-')

# Guardar archivo de datos

In [147]:
Final_res.to_excel(str(personas_test)[5:-1]+'_C='+str(numeroCarImp)+'_tri.xlsx')